In [1]:
# -*- coding: utf8 -*-
import requests
import lxml.html
from lxml import etree
import pandas as pd
import matplotlib.pyplot as plt
import jieba as jb
import jieba.posseg as pseg
import numpy as np
import re

In [2]:
# save original html to read on sublime and find pattern. do it when xpath is invalid or 
#doesn't have pattern across dif pages
#f = open("output.html","w")
#f.write(res.text.encode("utf-8"))
#f.close()

In [2]:
def getstores(urls):
    dic_sn = {}
    dic_sn["name"] = []
    dic_sn["rvs"] = []
    dic_sn["urls"] = []
    dic_sn["fitpro"] = []
    sn = []
    su = []
    sr = []
    sf = []
    for url in urls:
        res = requests.get(url)
        doc = etree.HTML(res.text)
        sn += doc.xpath("//table/tbody/*/td[1]/a/text()")
        su += doc.xpath("//table/tbody/*/td[1]/a/@href")
        sr += doc.xpath("//tbody/*/td[1]/p[2]/text()")
        sf += doc.xpath("//table/tbody/*/td/p[1]/a/em/text()")
    #take out duplicate urls
    su2 = []
    for (i,u) in enumerate(su):
        if i%2 ==0:
            u = re.sub(r'[^\*]+\*',r'',u)
            su2.append(u)
    #for url in su2:
        #res = requests.get(url)
        #doc = etree.HTML(res.text)
        
    for i, u, r, f in zip(sn, su2, sr, sf):
            dic_sn["name"].append(i)
            dic_sn["urls"].append(u)
            dic_sn["rvs"].append(r)
            dic_sn["fitpro"].append(f)
    return dic_sn

In [265]:
#get num of rvs and clean des
def orgdf(sdf):
    dic_new ={}
    dic_new["rvnb"] = []
    dic_new["n_nm"] = []
    dic_new["n_des"] = []
    #get number of rv out of string
    for i in range(len(sdf["rvs"])):
        dic_new["rvnb"].append([int(s) for s in sdf["rvs"][i].split() if s.isdigit()])
    for i,k in enumerate(dic_new["rvnb"]):
        dic_new["rvnb"][i] = re.sub('(\[)', '', str(k))
    for i,k in enumerate(dic_new["rvnb"]):
        dic_new["rvnb"][i] = re.sub('(\])', '', str(k))
    for i,k in enumerate(dic_new["rvnb"]):
        dic_new["rvnb"][i] = int(k)
    for i,k in enumerate(sdf["fitpro"]):
        sdf["fitpro"][i] = int(k)
    #clean des and add two extra cols
    try1 = sdf["name"].iloc[0:].tolist()
    for i in try1:
        dic_new["n_nm"].append(i.split(' - ', 1)[0])
        dic_new["n_des"].append(i.split(' - ', 1)[-1])
    for i, k in enumerate(dic_new["n_des"]):
        if k.endswith("..."):
            dic_new["n_des"][i] = k[:-3]
    new_df = pd.DataFrame(dic_new)
    n_m_df = pd.concat([new_df, sdf], axis = 1) 
    n_m_df = n_m_df.drop(["name", "rvs"], 1)
    #reoder cols
    n_m_df = n_m_df[["n_nm", "n_des", "fitpro", "rvnb", "urls"]]
    return n_m_df
    

In [263]:
# page order by num of rating for male
m_url = ["https://tw.mall.yahoo.com/152982021-merchant_category.html?catid=152982021&searchby=sname&apg=" + str(i+1) for i in range(17)]
s = '&sort_by=ratingpeople&order_by=0'
m_urls = [i + s for i in m_url]


In [5]:
# page order by num of rating for female
f_url = ["https://tw.mall.yahoo.com/152981802-merchant_category.html?catid=152981802&searchby=sname&apg=" + str(i+1) for i in range(32)]
s = '&sort_by=ratingpeople&order_by=0'
f_urls = [i + s for i in f_url]

In [266]:
#get male stores
m_dic = getstores(m_urls)
m_df = pd.DataFrame(m_dic)
cp_m_df = orgdf(m_df)
len(cp_m_df)

728

In [267]:
cp_m_df

,n_nm,n_des,fitpro,rvnb,urls
0,小三美日,小三美日批發價販售各國最新保養品彩妝，保證正品。與世界同步掌握最新訊息，誠信可,1,25922,https://tw.mall.yahoo.com/store/s3
1,屈臣氏Watsons,購物滿600免運 。反詐騙「2不1求證」：不操作ATM、不透露信用,43,12468,https://tw.mall.yahoo.com/store/watsons
2,OrangeBear中大尺碼,OrangeBear以多變化的搭配風格，帶領女孩穿出個人時,1,8730,https://tw.mall.yahoo.com/store/orangebear168
3,OB嚴選品牌旗艦店,OB嚴選提供最時尚魅力的穿搭流行趨勢，讓每位女孩輕鬆玩時尚！！讓你輕,4,6565,https://tw.mall.yahoo.com/store/obdesign
4,小二‧布屋,引領韓風時尚品味，有型是我們的生活態度。精心細選出符合亞洲體型的剪裁每件商品,2322,6297,https://tw.mall.yahoo.com/store/a74226
5,康是美藥妝店,榮獲2015台灣服務業大評鑑：連鎖藥局／藥妝店金牌大賞！,24,6105,https://tw.mall.yahoo.com/store/cosmed
6,鴻墨水耗材專賣店,▲▲最新最流行3C週邊配件&創意小物，買不到的獨家商品我們通通都有▲▲,1,4851,https://tw.mall.yahoo.com/store/hung
7,巴黎草莓,巴黎草莓-專業美妝團隊，專門為愛漂亮的水水們，購入各國最新保養彩妝商品，一對一,1,4691,https://tw.mall.yahoo.com/store/fdp
8,ezmore購物網,我們販售的商品 皆為市場上流行以及當紅限定商品 歡迎批發團購04-,1,4665,https://tw.mall.yahoo.com/store/ezmore
9,player,引進美．日街頭 風格 商品，融合時尚、街頭、潮流等元素，賦予多樣化的搭配選,194,4463,https://tw.mall.yahoo.com/store/player-one


In [183]:
#get female stores
f_dic = getstores(f_urls)
f_df = pd.DataFrame(f_dic)
cp_f_df = orgdf(f_df)
len(cp_f_df)

1451

In [184]:
cp_f_df

,n_nm,n_des,fitpro,rvnb,urls
0,Miu-Star,▲冬現貨↘49折 133/件▲早春裝搶先上市！▲亮眼休閒為基調，融合鄰,1482,36761,https://tw.mall.yahoo.com/store/miu-star
1,小三美日,小三美日批發價販售各國最新保養品彩妝，保證正品。與世界同步掌握最新訊息，誠信可,102,25943,https://tw.mall.yahoo.com/store/s3
2,糖罐子,2015新風尚‧新色彩‧candy box提供水水們豐富的style，讓喜愛打扮,3169,22392,https://tw.mall.yahoo.com/store/jacky5318
3,東京著衣,全部現貨！急凍降溫5℃【抗寒外套↘任二件5折】★歲末回饋【冬品現貨↘5折】★週,2614,21877,https://tw.mall.yahoo.com/store/mayuki0920
4,86小舖,到86小舖不僅可以找到最新最流行的美妝產品，及別家買不到的獨家商品我們通通都有,35,15876,https://tw.mall.yahoo.com/store/86shops
5,漢神百貨,漢神百貨網路購物延續漢神百貨與漢神巨蛋的經營理念，線上萬件商品種類豐富，提供值,268,14534,https://tw.mall.yahoo.com/store/hanshin
6,LuLu’s,★全館599元免運費、現貨69折快速出貨!★因超商取貨有裁積限制，單筆訂單,605,12831,https://tw.mall.yahoo.com/store/lulus
7,屈臣氏Watsons,購物滿600免運 。反詐騙「2不1求證」：不操作ATM、不透露信用,141,12461,https://tw.mall.yahoo.com/store/watsons
8,嘉蒂斯,★NATURAL CHARM AND FASHION STYLE ：個性、甜美、,654,11102,https://tw.mall.yahoo.com/store/goddess168
9,OrangeBear中大尺碼,OrangeBear以多變化的搭配風格，帶領女孩穿出個人時,386,8738,https://tw.mall.yahoo.com/store/orangebear168


In [274]:
#plot rvn
m_p = plt.hist(cp_m_df["rvnb"], bins = 200)
plt.show(m_p)
f_p = plt.hist(cp_m_df["rvnb"], bins = 200)
plt.show(f_p)

In [275]:
#plot fitpro
m_p = plt.hist(cp_m_df["fitpro"], bins = 200)
plt.show(m_p)
f_p = plt.hist(cp_m_df["fitpro"], bins = 200)
plt.show(f_p)

In [277]:
#set rules for rvn segments
def rv_lv(c):
    if c['rvnb'] >= 10000:
        return 'a:~10,000'
    elif 1000 <= c['rvnb'] < 10000:
        return 'b:10,000~1,000'
    elif 100 <= c['rvnb'] < 1000:
        return 'c:1,000~100'
    elif 10 <= c['rvnb'] < 100:
        return 'd:100~10'
    elif 1 <= c['rvnb'] < 10:
        return 'e:10~1'
    elif c['rvnb'] == 0:
        return 'f:0'

In [320]:
#set rules for fitpro segments
def fp_lv(c):
    if c['fitpro'] >= 10000:
        return 'A:~10,000'
    elif 1000 <= c['fitpro'] < 10000:
        return 'B:10,000~1,000'
    elif 100 <= c['fitpro'] < 1000:
        return 'C:1,000~100'
    elif 10 <= c['fitpro'] < 100:
        return 'D:100~10'
    elif 1 <= c['fitpro'] < 10:
        return 'E:10~1'
    elif c['fitpro'] == 0:
        return 'F:0'

In [279]:
#give new segments to data 
cp_m_df['rvlv'] = cp_m_df.apply(rv_lv, axis=1)
cp_f_df['rvlv'] = cp_f_df.apply(rv_lv, axis=1)

In [321]:
#give new segments to data 
cp_m_df['fplv'] = cp_m_df.apply(fp_lv, axis=1)
cp_f_df['fplv'] = cp_f_df.apply(fp_lv, axis=1)

In [283]:
# count table for rvlv
cp_m_df[["rvnb","rvlv"]].groupby(['rvlv']).agg(['mean', 'count']) 

rvnb      
                        mean count
rvlv                              
a:~10,000       19308.555556     9
b:10,000~1,000   2337.054688   128
c:1,000~100       341.678937   489
d:100~10           45.108856   542
e:10~1              3.975728   206
f:0                 0.000000    77

In [284]:
cp_f_df[["rvnb","rvlv"]].groupby(['rvlv']).agg(['mean', 'count']) 

rvnb      
                        mean count
rvlv                              
a:~10,000       19308.555556     9
b:10,000~1,000   2337.054688   128
c:1,000~100       341.678937   489
d:100~10           45.108856   542
e:10~1              3.975728   206
f:0                 0.000000    77

In [323]:
cp_m_df

,n_nm,n_des,fitpro,rvnb,urls,rvlv,fplv
0,小三美日,小三美日批發價販售各國最新保養品彩妝，保證正品。與世界同步掌握最新訊息，誠信可,1,25922,https://tw.mall.yahoo.com/store/s3,"a:~10,000",E:10~1
1,屈臣氏Watsons,購物滿600免運 。反詐騙「2不1求證」：不操作ATM、不透露信用,43,12468,https://tw.mall.yahoo.com/store/watsons,"a:~10,000",D:100~10
2,OrangeBear中大尺碼,OrangeBear以多變化的搭配風格，帶領女孩穿出個人時,1,8730,https://tw.mall.yahoo.com/store/orangebear168,"b:10,000~1,000",E:10~1
3,OB嚴選品牌旗艦店,OB嚴選提供最時尚魅力的穿搭流行趨勢，讓每位女孩輕鬆玩時尚！！讓你輕,4,6565,https://tw.mall.yahoo.com/store/obdesign,"b:10,000~1,000",E:10~1
4,小二‧布屋,引領韓風時尚品味，有型是我們的生活態度。精心細選出符合亞洲體型的剪裁每件商品,2322,6297,https://tw.mall.yahoo.com/store/a74226,"b:10,000~1,000","B:10,000~1,000"
5,康是美藥妝店,榮獲2015台灣服務業大評鑑：連鎖藥局／藥妝店金牌大賞！,24,6105,https://tw.mall.yahoo.com/store/cosmed,"b:10,000~1,000",D:100~10
6,鴻墨水耗材專賣店,▲▲最新最流行3C週邊配件&創意小物，買不到的獨家商品我們通通都有▲▲,1,4851,https://tw.mall.yahoo.com/store/hung,"b:10,000~1,000",E:10~1
7,巴黎草莓,巴黎草莓-專業美妝團隊，專門為愛漂亮的水水們，購入各國最新保養彩妝商品，一對一,1,4691,https://tw.mall.yahoo.com/store/fdp,"b:10,000~1,000",E:10~1
8,ezmore購物網,我們販售的商品 皆為市場上流行以及當紅限定商品 歡迎批發團購04-,1,4665,https://tw.mall.yahoo.com/store/ezmore,"b:10,000~1,000",E:10~1
9,player,引進美．日街頭 風格 商品，融合時尚、街頭、潮流等元素，賦予多樣化的搭配選,194,4463,https://tw.mall.yahoo.com/store/player-one,"b:10,000~1,000","C:1,000~100"


In [328]:
######### PROBLEM ###########
# count table for fplv
#cp_m_df[["fitpro","fplv"]].groupby(['fplv']).agg(['mean', 'count']) 
#cp_m_df["fplv"].sort_values()

In [292]:
#cp_f_df[["fitpro","fplv"]].groupby(['fplv']).agg(['mean', 'count']) 

In [330]:
# cut sentences
jb.set_dictionary('dict.txt.big')

In [331]:
# cut sentences and make count table
def cutst(s):
    wf = {}
    wf["wdic"] = []
    wf["fdic"] = []
    for i in s['n_des']:
        #method with word class
        cut = pseg.cut(i)
        for w in cut:
            wf["wdic"].append(w.word)
            wf["fdic"].append(w.flag)
    wf_df = pd.DataFrame(wf)
    #make a count table
    grouped = wf_df.groupby(['wdic', 'fdic'], as_index=False)
    pw = pd.DataFrame(grouped.size())
    #put wrong index to new column, need to run twice for two wrong indexes
    pw.reset_index(level=0, inplace=True)
    pw.reset_index(level=0, inplace=True)
    pw = pw.rename(columns = {0:'fq'})
    return pw

In [332]:
m_wcrs = cutst(cp_m_df)

Building prefix dict from /Users/Charlie/Crawler/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /Users/Charlie/Crawler/dict.txt.big ...
Dumping model to file cache /var/folders/k8/576_7c3d27d9_cgc_vv_ylqw0000gn/T/jieba.u65c5d577a1e955573193dc1069ffe005.cache
DEBUG:jieba:Dumping model to file cache /var/folders/k8/576_7c3d27d9_cgc_vv_ylqw0000gn/T/jieba.u65c5d577a1e955573193dc1069ffe005.cache
Loading model cost 2.995 seconds.
DEBUG:jieba:Loading model cost 2.995 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [339]:
f_wcrs = cutst(cp_f_df)

In [341]:
#select only noun and fq > 10
m_wc_n = m_wcrs[(m_wcrs["fdic"] == "n") & (m_wcrs["fq"] > 10)]
#m_wc_n.to_csv("m_wc.csv", encoding='utf-8')
#m_wc_n


In [342]:
#select only noun and fq > 10
f_wc_n = f_wcrs[(f_wcrs["fdic"] == "n") & (f_wcrs["fq"] > 10)]
#f_wc_n.to_csv("f_wc.csv", encoding='utf-8')
#f_wc_n

In [343]:
#童裝 情趣用品 內衣
m_df_f = cp_m_df[~cp_m_df['n_des'].str.contains(u'情趣用品|內衣|童裝|小三美日|屈臣氏')]

In [344]:
#  童裝 情趣用品 男裝 手機 寢具 玩具 美妝 內衣 雜貨
f_df_f = cp_f_df[~cp_f_df['n_des'].str.contains(u'情趣用品|童裝|內衣|寢具|手機|玩具|美妝|雜貨|小三美日|屈臣氏')]

In [ ]:
f_df_f[f_df_f["rvlv"] == "a:~10,000"].sample()

In [354]:
f_df_f["rvlv"][f_df_f["rvlv"] == "a:~10,000"].count()/ float(f_df_f["rvlv"].count())

0.0053151100987091872

In [375]:
f_df_f["rvlv"].()

AttributeError: 'Series' object has no attribute 'level'

In [ ]:
def getsp(df, n):
    lv = ["a:~10,000", "b:10,000~1,000", "c:1,000~100", "e:10~1", "f:0"]
    for i in lv:
        pt = n * (df["rvlv"][df["rvlv"] == "a:~10,000"].count()/ float(df["rvlv"].count()))
    a_p = n * (df["rvlv"][df["rvlv"] == "a:~10,000"].count()/ float(df["rvlv"].count()))
    b_p = n * (df["rvlv"][df["rvlv"] == "b:10,000~1,000"].count()/ float(df["rvlv"].count()))
    c_p = n *df["rvlv"][df["rvlv"] == "c:1,000~100"].count()/ float(df["rvlv"].count())
    d_p = df["rvlv"][df["rvlv"] == "d:100~10"].count()/ float(df["rvlv"].count())
    e_p = df["rvlv"][df["rvlv"] == "e:10~1"].count()/ float(df["rvlv"].count())
    f_p = df["rvlv"][df["rvlv"] == "f:0"].count()/ float(df["rvlv"].count())
    n*a_p

In [373]:
a=13.5

In [374]:
print("%.f" % a)

14


In [358]:
print("%.3f" % a)

13.946


In [377]:
#if "..." in raws:
#    ss = re.findall(r"(?<=-\s).*(?=\...)", raws)
#    if ss = 
#else:
#    ss = re.findall(r"(?:-\s)(.*)", raws)
#to make it work for code below
#idx = res_new.find(ss)
#idx2 = res_new[idx:].find('"><')
#comdes = res_new[idx:idx+idx2]
#comname.append(comdes)
#get full des
def getstoredes(df):
    #from mobile page to normal page
    __HEADERS = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/27.0.1453.93 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
    'Accept-Encoding': 'gzip,deflate,sdch'
    }
    cp = {}
    cp["cp_des"] = []
    urls = df["urls"].iloc[0:44].tolist()
    for (i,url) in enumerate(urls):
        res = requests.get(url, headers = __HEADERS)
        doc = etree.HTML(res.text)
        res_new = res.text.replace("\n","")
        #turn df to list
        raws = df["n_des"].iloc[[i]].tolist()
        #turn list to string
        raws = "".join(raws)
        #match with first 2 and last 2 
        ss = re.findall(ur">([^>]{0,2}%s[^<]*%s[^<]*)<" % (raws[:2],raws[-2:]), res_new)
        ss = "".join(ss)
        
        cp["cp_des"].append(ss)
    cpdf1 = pd.DataFrame(cp)
    cpdf2 = pd.concat([cpdf1, df], axis = 1)
    cpdf2 = cpdf2[["n_nm", "n_des", "cp_des", "fitpro", "rvnb", "urls"]]
    return cpdf2

In [378]:
fn_m_df = getstoredes(cp_m_df)

error: unbalanced parenthesis

AttributeError: 'module' object has no attribute 'escpae'

In [379]:
fn_m_df[0:44]

,n_nm,n_des,cp_des,fitpro,rvnb,urls
0,小三美日,小三美日批發價販售各國最新保養品彩妝，保證正品。與世界同步掌握最新訊息，誠信可,小三美日批發價販售各國最新保養品彩妝，保證正品。與世界同步掌握最新訊息，誠信可靠，快速寄送，...,1,25943,https://tw.mall.yahoo.com/store/s3
1,屈臣氏Watsons,購物滿600免運 。反詐騙「2不1求證」：不操作ATM、不透露信用,購物滿600免運 。反詐騙「2不1求證」：不操作ATM、不透露信用卡資料、警政署防詐騙專線1...,43,12461,https://tw.mall.yahoo.com/store/watsons
2,OrangeBear中大尺碼,OrangeBear以多變化的搭配風格，帶領女孩穿出個人時,OrangeBear以多變化的搭配風格，帶領女孩穿出個人時尚，綻放自我迷人魅力；以高質感、平...,1,8738,https://tw.mall.yahoo.com/store/orangebear168
3,OB嚴選品牌旗艦店,OB嚴選提供最時尚魅力的穿搭流行趨勢，讓每位女孩輕鬆玩時尚！！讓你輕,OB嚴選提供最時尚魅力的穿搭流行趨勢，讓每位女孩輕鬆玩時尚！！讓你輕鬆駕馭時尚、操縱流行、玩...,4,6566,https://tw.mall.yahoo.com/store/obdesign
4,小二‧布屋,引領韓風時尚品味，有型是我們的生活態度。精心細選出符合亞洲體型的剪裁每件商品,引領韓風時尚品味，有型是我們的生活態度。精心細選出符合亞洲體型的剪裁每件商品，設計專屬於您的...,2323,6316,https://tw.mall.yahoo.com/store/a74226
5,康是美藥妝店,榮獲2015台灣服務業大評鑑：連鎖藥局／藥妝店金牌大賞！,榮獲2015台灣服務業大評鑑：連鎖藥局／藥妝店金牌大賞！,24,6107,https://tw.mall.yahoo.com/store/cosmed
6,鴻墨水耗材專賣店,▲▲最新最流行3C週邊配件&創意小物，買不到的獨家商品我們通通都有▲▲,▲▲最新最流行3C週邊配件&創意小物，買不到的獨家商品我們通通都有▲▲專業團隊為您把關品質及...,1,4849,https://tw.mall.yahoo.com/store/hung
7,巴黎草莓,巴黎草莓-專業美妝團隊，專門為愛漂亮的水水們，購入各國最新保養彩妝商品，一對一,巴黎草莓-專業美妝團隊，專門為愛漂亮的水水們，購入各國最新保養彩妝商品，一對一客服，貼心服務...,1,4706,https://tw.mall.yahoo.com/store/fdp
8,ezmore購物網,我們販售的商品 皆為市場上流行以及當紅限定商品 歡迎批發團購04-,我們販售的商品 皆為市場上流行以及當紅限定商品 歡迎批發團購04-3700048804-3...,1,4674,https://tw.mall.yahoo.com/store/ezmore
9,player,引進美．日街頭 風格 商品，融合時尚、街頭、潮流等元素，賦予多樣化的搭配選,引進美．日街頭 風格 商品，融合時尚、街頭、潮流等元素，賦予多樣化的搭配選擇。本館引進美．日...,194,4481,https://tw.mall.yahoo.com/store/player-one


In [104]:
#keep it. get html to sublime to check pattern
__HEADERS = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/27.0.1453.93 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
    'Accept-Encoding': 'gzip,deflate,sdch'
    }
url_test = "https://tw.mall.yahoo.com/store/osmar"
res = requests.get(url_test, headers = __HEADERS)
f = open("output_test.html", "w")
f.write(res.text.encode("utf-8"))
f.close()